In [1]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
import numpy as np
import faiss
from pathlib import Path
import os
import re
import pickle
from tqdm import tqdm

In [2]:
markdown_path = Path("./postgre_chunks/accumulate.md")
markdown_text = markdown_path.read_text(encoding="utf-8")


In [3]:
model = SentenceTransformer(
    "jinaai/jina-embeddings-v2-small-en",
    trust_remote_code=True
)

doc_embeddings = model.encode(markdown_text, normalize_embeddings=True)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

C:\Users\adika\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\adika\.cache\huggingface\hub\models--jinaai--jina-embeddings-v2-small-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/70.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
doc_embeddings

array([-3.93222310e-02, -6.93455786e-02,  5.24517987e-03,  6.72828034e-02,
       -3.10698040e-02,  5.02861803e-03,  2.64600459e-02,  6.33088797e-02,
       -1.60452891e-02, -7.57627364e-04,  3.93325351e-02, -2.56419145e-02,
       -1.02493865e-02, -8.09574276e-02,  5.36529906e-02, -5.79424351e-02,
        5.35768047e-02, -7.13758394e-02, -5.50481193e-02, -5.89387231e-02,
       -5.03211021e-02, -3.31346169e-02,  5.08580804e-02, -9.79123358e-03,
        7.36794621e-02, -3.65940332e-02, -4.22500558e-02,  4.82253768e-02,
        2.00222265e-02,  5.69430254e-02, -1.29322410e-01,  2.66738124e-02,
       -1.18071325e-02,  3.34186368e-02, -5.42167164e-02,  3.75753804e-03,
        2.35779919e-02,  4.65796590e-02, -4.21759263e-02, -8.36880952e-02,
       -4.89322990e-02,  2.53802221e-02,  6.59107566e-02,  1.54676745e-02,
       -5.81718311e-02,  3.65538374e-02, -7.91455507e-02,  1.07679911e-01,
        2.29995344e-02, -3.49104498e-03, -1.09366402e-01,  2.99628396e-02,
       -8.32658261e-02,  